In [ ]:
import stata_setup, os
if os.name == 'nt':
    stata_setup.config('C:/Program Files/Stata17/','mp')
else:
    stata_setup.config('/usr/local/stata17','mp')

Loading the data set

In [ ]:
import pandas as pd
import numpy as np
cps09mr = pd.read_stata('../Data/cps09mar.dta')

Constructing the relevant variables

In [ ]:
cps09mr['wage'] = np.log(cps09mr.earnings/(cps09mr.hours*cps09mr.week))
cps09mr['experience'] = cps09mr.age - cps09mr.education - 6
cps09mr['exp2'] = (cps09mr.experience**2)/100

Filtering data based on marital status, race and gender

In [ ]:
cps09mr['mnwf'] = np.logical_and(np.logical_and(cps09mr.marital<=2, cps09mr.race!=1), cps09mr.female==1)

Performing the OLS regression of $\boldsymbol{Y}$ on $\boldsymbol{X}$ using ```Stata```:

In [ ]:
%%stata -d cps09mr -force
qui reg wage education experience exp2 if mnwf==1

Printing the OLS estimates $\widehat{\beta}$

In [ ]:
%stata matrix list e(b)

Printing the $TSS$

In [ ]:
%stata display e(mss)+e(rss)

Printing the $ESS$

In [ ]:
%stata display e(mss)

Printing the $RSS$

In [ ]:
%stata display e(rss)

Printing the $R^2$

In [ ]:
%stata display e(mss)/(e(mss)+e(rss))

# Leverage Values & LOO Regression

In [ ]:
from statsmodels.regression.linear_model import OLS
from statsmodels.stats.outliers_influence import OLSInfluence
import patsy

Creating the training data, i.e., $\boldsymbol{Y}$ vector and $\boldsymbol{X}$ matrix

In [ ]:
f = 'wage ~ education + experience + exp2 '
y, X = patsy.dmatrices(f, data=cps09mr.loc[cps09mr['mnwf']], return_type='dataframe')

Performing the OLS regression of $\boldsymbol{Y}$ on $\boldsymbol{X}$ using ```Python```:

In [ ]:
fit = OLS(y, X).fit()

Extracting the leverage values

In [ ]:
hii = OLSInfluence(fit).hat_matrix_diag

Checking that $0\le h_{ii} \le 1$

In [ ]:
print([hii.min(),hii.max()])

Checking that $h_{ii}\ge 1/n$ by checking that $\min_{i=1,\ldots,n}(h_{ii})\ge 1/n$

In [ ]:
(hii.min()>=1/hii.size)

Checking that $\sum_{i=1}^n h_{ii}=k$

In [ ]:
hii.sum().round().astype(int)

Extracting the matrix $\widehat{\beta}_{(-i)}$ and checking its dimension

In [ ]:
bi = OLSInfluence(fit).dfbetas
print(bi.shape)

Extracting the LOO residuals $\widetilde{e}_{i}$ and check its dimension

In [ ]:
e_tilde = OLSInfluence(fit).resid_press
print(e_tilde.size)

Plotting the sets $\{\widehat{\beta}_{(-i)}:i=1,\ldots,n\}$ and $\{\widetilde{e}_{i}:i=1,\ldots,n\}$ against their indexes $i=1,\ldots,n$

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(2)
axs[0].plot(np.linspace(1,e_tilde.size,num=e_tilde.size).astype(int), bi[:,1])
axs[0].set_title(r'$\widehat{\beta}_{-i}$ vs $i=1,\ldots,n$')
axs[1].plot(np.linspace(1,e_tilde.size,num=e_tilde.size).astype(int), e_tilde,'tab:orange')
axs[1].set_title(r'$\widetilde{e}_{i}$ vs $i=1,\ldots,n$')
# Hide x labels and tick labels for top plots
for ax in axs.flat:
    ax.label_outer()

The '*prediction standard error*' is:

In [ ]:
print(np.sqrt(np.mean(e_tilde**2)))

## Residuals

In [ ]:
residuos = pd.DataFrame({'e_hat': fit.resid, 'e_tilde': e_tilde, 'e_bar': fit.resid/np.sqrt(1-hii)})
print(residuos)

In [ ]:
from pystata import stata
stata.pdataframe_to_data(residuos, force=True)
stata.run('summarize')

## Covariance Matrix Estimation

**<span style="color:red">Homoskedasticity:</span>**

$$
\widehat{\mathbf{V}}_{\widehat{\beta}}^{0}=\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1} s^{2}
$$

In [ ]:
print(fit.cov_params())

In [ ]:
from pystata import stata
stata.pdataframe_to_data(cps09mr, force=True)
stata.run('''
qui reg wage education experience exp2 if mnwf==1
''',quietly=True)
stata.run('matrix list e(V)')

**<span style="color:red">HC0:</span>**

$$
\widehat{\mathbf{V}}_{\widehat{\beta}}^{\mathrm{HC0}}=\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}\left(\sum_{i=1}^{n} X_{i} X_{i}^{\prime}\widehat{e}_{i}^{2} \right)\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}
$$

In [ ]:
print(fit.cov_HC0)

**<span style="color:red">HC1:</span>** (most common in *econometrics*)

$$
\widehat{\mathbf{V}}_{\widehat{\beta}}^{\mathrm{HCl}}=\left(\frac{n}{n-k}\right)\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}\left(\sum_{i=1}^{n} X_{i} X_{i}^{\prime}\widehat{e}_{i}^{2} \right)\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}
$$


In [ ]:
print(fit.cov_HC1)

In [ ]:
stata.run('''
qui reg wage education experience exp2 if mnwf==1, robust
''',quietly=True)
stata.run('matrix list e(V)')

**<span style="color:red">HC2:</span>**

$$
\widehat{\mathbf{V}}_{\widehat{\beta}}^{\mathrm{HC2}}=\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}\left(\sum_{i=1}^{n} \left(1-h_{i i}\right)^{-1} X_{i}X_{i}^{\prime}\widehat{e}_{i}^{2} \right)\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}
$$

In [ ]:
print(fit.cov_HC2)

In [ ]:
stata.run('''
qui reg wage education experience exp2 if mnwf==1, vce(hc2)
''',quietly=True)
stata.run('matrix list e(V)')

**<span style="color:red">HC3:</span>**

$$
\widehat{\mathbf{V}}_{\widehat{\beta}}^{\mathrm{HC3}}=\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}\left(\sum_{i=1}^{n}  \left(1-h_{i i}\right)^{-2}X_{i}X_{i}^{\prime}\widehat{e}_{i}^{2}  \right)\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}
$$

In [ ]:
print(fit.cov_HC3)

In [ ]:
stata.run('''
qui reg wage education experience exp2 if mnwf==1, vce(hc3)
''',quietly=True)
stata.run('matrix list e(V)')

## Standard Errors

**<span style="color:red">Homoskedasticity:</span>**

In [ ]:
print(fit.bse)

**<span style="color:red">HC0:</span>**

In [ ]:
print(fit.HC0_se)

**<span style="color:red">HC1:</span>**

In [ ]:
print(fit.HC1_se)

**<span style="color:red">HC2:</span>**

In [ ]:
print(fit.HC2_se)

**<span style="color:red">HC3:</span>**

In [ ]:
print(fit.HC3_se)

## Measures of Fit

**<span style="color:red">R-squared:</span>**
$$
R^{2}=1-\frac{\sum_{i=1}^{n} \widehat{e}_{i}^{2}}{\sum_{i=1}^{n}\left(y_{i}-\bar{y}\right)^{2}}
$$

In [ ]:
print(fit.rsquared)

In [ ]:
from sfi import Scalar
rsquared=Scalar.getValue('e(r2)')
print(rsquared)

**<span style="color:red">Adjusted R-squared:</span>**

$$
\bar{R}^{2}=1-\frac{(n-1) \sum_{i=1}^{n} \hat{e}_{i}^{2}}{(n-k) \sum_{i=1}^{n}\left(y_{i}-\bar{y}\right)^{2}}
$$

In [ ]:
print(fit.rsquared_adj)

In [ ]:
radj=Scalar.getValue('e(r2_a)')
print(radj)

**<span style="color:red">(Alternative) R-squared:</span>**

$$
\widetilde{R}^{2}=1-\frac{\sum_{i=1}^{n} \widetilde{e}_{i}^{2}}{\sum_{i=1}^{n}\left(y_{i}-\bar{y}\right)^{2}}
$$

In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence
R2_tilde = 1 - (OLSInfluence(fit).resid_press**2).sum()/fit.centered_tss
print(R2_tilde)